In [9]:
import pandas as pd
import matplotlib.pyplot as plt

data = pd.read_excel("responses.xlsx")
data = data[~data['Remove PII'].isna()]

error_columns = ['Remove PII', 'Symptoms, Reason to call, Rec', 'Summarized']

In [16]:
data[error_columns] = data[error_columns].astype('bool')

### Error rates for each node

In [18]:
def error_rate_for_node(df, node_name):
    num_rows = len(df)
    return df[node_name].value_counts(normalize=True)
    

In [23]:
print(f"PII node error rate: {error_rate_for_node(data, node_name=error_columns[0])}")

PII node error rate: Remove PII
True     0.565217
False    0.434783
Name: proportion, dtype: float64


#### 43% of the data had PII information. This is a problematic node.

In [32]:
print(f"Main Reason node error rate: {error_rate_for_node(data, node_name=error_columns[1])}")

Main Reason node error rate: Symptoms, Reason to call, Rec
True     0.724638
False    0.275362
Name: proportion, dtype: float64


In [31]:
print(f"Summarizer node error rate: {error_rate_for_node(data, node_name=error_columns[2])}")

Summarizer node error rate: Summarized
True     0.768116
False    0.231884
Name: proportion, dtype: float64


In [45]:
data['all_three_correct'] = data[error_columns[0]] | data[error_columns[1]] | data[error_columns[2]] 

data['node_2_and_3_incorrect'] = data[error_columns[1]] | data[error_columns[2]]

In [44]:
data['all_three_correct'].value_counts(normalize=True)

all_three_correct
True     0.826087
False    0.173913
Name: proportion, dtype: float64

#### Seems like 17% of the time all three of the nodes breakdown. That could be because PII leaked through the chain, or that the symptoms or main reason for the call was off in addition to PII.

In [46]:
data['node_2_and_3_incorrect'].value_counts(normalize=True)

node_2_and_3_incorrect
True     0.782609
False    0.217391
Name: proportion, dtype: float64

#### Node 2 and 3 are certainly linked and their errors move together

In [60]:
data[data['node_2_and_3_incorrect'] == False]['Notes']

4      PII and main reason. As a result summary is off.
5               PII leaked into recs through the chain 
9      PII and main reason. As a result summary is off.
12    PII and main reason. Summary also doesn't capt...
19              PII leaked into recs through the chain 
33     PII and main reason. As a result summary is off.
35    PII and main reason. Summary also doesn't capt...
36    PII and main reason. Summary also doesn't capt...
45               PII, "some symptoms", summary off too 
47                                                  NaN
49              PII leaked into recs through the chain 
53              PII leaked into recs through the chain 
55    Main reason for the call leaked through to the...
60     PII and main reason. As a result summary is off.
68    Main reason for the call leaked through to the...
Name: Notes, dtype: object

#### Seems like there are instances where PII leaks through the entire chain. But more commonly if the main reason is wrong, the summary is wrong.